In [1]:
!pip3 install torch
!pip3 install datasets transformers seqeval
!pip install accelerate -U
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.0 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=e460405489d3c3aa39ae59bd3885fbc6c3e37aff9366de2d816622fa78faa8e2
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.0 MB/s eta 0:00:00


In [2]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-cu0rcbh0
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-cu0rcbh0
  Resolved https://github.com/huggingface/transformers to commit d6392482bd8af0d28903b5531ed04bd7273931a3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8148049 sha256=e805d93794c7ab75a34e8c8be8c2fc72252efa576761296ca13a5d29e323186d
  Stored in directory: /tmp/pip-ephem-wheel-cache-3x79z4ov/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [84]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.


In [47]:
import json
import numpy as np
import pandas as pd
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, AutoTokenizer, AdamW
import torch
import datasets
from datasets import load_dataset, load_metric
from torch.utils.data import Dataset, DataLoader, Subset
from tqdm.notebook import tqdm
import transformers
import random

print(transformers.__version__)

4.36.0.dev0


In [6]:
task = "ner"
model_checkpoint = "distilbert-base-uncased"
batch_size = 8

In [7]:
dataset = load_dataset("Babelscape/multinerd")

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [8]:
type(dataset["test"])

datasets.arrow_dataset.Dataset

Create a shuffled subset with the code below for smaller size of data. (Optional)

In [61]:
train_data_list = list(dataset['train'])
validation_data_list = list(dataset['validation'])
test_data_list = list(dataset['test'])

random.shuffle(train_data_list)
random.shuffle(validation_data_list)
random.shuffle(test_data_list)

sample_size = 0.1

small_train_size = int(sample_size * len(train_data_list))
small_validation_size = int(sample_size * len(validation_data_list))
small_test_size = int(sample_size * len(test_data_list))

small_train_list = train_data_list[:small_train_size]
small_validation_list = validation_data_list[:small_validation_size]
small_test_list = test_data_list[:small_test_size]

sampled_train = dataset['train'].from_pandas(pd.DataFrame(small_train_list))
sampled_validation = dataset['validation'].from_pandas(pd.DataFrame(small_validation_list))
sampled_test = dataset['test'].from_pandas(pd.DataFrame(small_test_list))

Check the sampled dataset size.

In [68]:
len(sampled_train), len(sampled_validation), len(sampled_test)

(267840, 33480, 33598)

In [9]:
class dataset_preprocessor:
    def __init__(self, data):
        self.data = data

    def extract_lang(self,language):
        self.language = language
        self.extracted_lang_data = []
        for l in self.language:
            for i, e in enumerate(self.data['lang']):
                self.lang_dataset= {'tokens':[], 'ner_tags':[], 'lang':[]}
                if e == l:
                    self.lang_dataset['tokens'] = self.data[i]['tokens']
                    self.lang_dataset['ner_tags']= self.data[i]['ner_tags']
                    self.lang_dataset['lang'] = e
                    self.extracted_lang_data.append(self.lang_dataset)
        return self.extracted_lang_data

    def extract_tags(self, tags, tags_index_dict):
        self.tags = tags #list of wanted tags
        self.tags_index_dict = tags_index_dict
        self.extracted_tag_data = []
        self.indices = []

        for t in self.tags:
            if t == 'O':
                self.indices.append(self.tags_index_dict[t])
            else:
                self.indices.append(self.tags_index_dict['B-' + t])
                self.indices.append(self.tags_index_dict['I-' + t])
        for i,e in enumerate(self.data['ner_tags']):
            self.tags_dataset= {'tokens':[], 'ner_tags':[], 'lang':[]}
            self.new_tags = []
            for each_tag in e:
                if each_tag in self.indices:
                    self.new_tags.append(each_tag)
                elif each_tag not in self.indices:
                    self.new_tags.append(0)
            self.tags_dataset['tokens'] = self.data[i]['tokens']
            self.tags_dataset['ner_tags']= self.new_tags
            self.tags_dataset['lang']= self.data[i]['lang']
            self.extracted_tag_data.append(self.tags_dataset)
        return self.extracted_tag_data

Run the code below to get only English data for system A.

In [10]:
lang_preprocess_train = dataset_preprocessor(dataset['train'])
lang_preprocess_validation = dataset_preprocessor(dataset['validation'])
lang_preprocess_test= dataset_preprocessor(dataset['test'])

lang_wanted = ['en']

en_train = lang_preprocess_train.extract_lang(lang_wanted)
en_validation = lang_preprocess_validation.extract_lang(lang_wanted)
en_test = lang_preprocess_test.extract_lang(lang_wanted)

Run the code below to get only 5 NER tags data for system B. Due to the limited resources, data is instanciated from sampled dataset. Replace (*sampled_train*) with (*dataset["train"]*) to use the full size data. Same with validation and test.

In [69]:
tag_preprocess_train = dataset_preprocessor(sampled_train)
tag_preprocess_validation = dataset_preprocessor(sampled_validation)
tag_preprocess_test = dataset_preprocessor(sampled_test)

wanted_tags =  ['PER','ORG', 'LOC', 'DIS', 'ANIM']
tag_index = {
    "O": 0,
    "B-PER": 1,
    "I-PER": 2,
    "B-ORG": 3,
    "I-ORG": 4,
    "B-LOC": 5,
    "I-LOC": 6,
    "B-ANIM": 7,
    "I-ANIM": 8,
    "B-BIO": 9,
    "I-BIO": 10,
    "B-CEL": 11,
    "I-CEL": 12,
    "B-DIS": 13,
    "I-DIS": 14,
    "B-EVE": 15,
    "I-EVE": 16,
    "B-FOOD": 17,
    "I-FOOD": 18,
    "B-INST": 19,
    "I-INST": 20,
    "B-MEDIA": 21,
    "I-MEDIA": 22,
    "B-MYTH": 23,
    "I-MYTH": 24,
    "B-PLANT": 25,
    "I-PLANT": 26,
    "B-TIME": 27,
    "I-TIME": 28,
    "B-VEHI": 29,
    "I-VEHI": 30,
  }

In [70]:
tag_train = tag_preprocess_train.extract_tags(wanted_tags, tag_index)
tag_validation = tag_preprocess_validation.extract_tags(wanted_tags, tag_index)
tag_test = tag_preprocess_test.extract_tags(wanted_tags, tag_index)

Code below convert the preprocessed data from lists to Dataset objects which is required to run the model.

In [13]:
en_train_set = datasets.Dataset.from_pandas(pd.DataFrame(data=en_train))
en_validation_set = datasets.Dataset.from_pandas(pd.DataFrame(data=en_validation))
en_test_set = datasets.Dataset.from_pandas(pd.DataFrame(data=en_test))

In [74]:
tag_train_set = datasets.Dataset.from_pandas(pd.DataFrame(data=tag_train))
tag_validation_set = datasets.Dataset.from_pandas(pd.DataFrame(data=tag_validation))
tag_test_set = datasets.Dataset.from_pandas(pd.DataFrame(data=tag_test))

Define a tokenizer.

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [16]:
# example = en_test_set[1]
# print(example['tokens'])

In [17]:
# tokenized_input=tokenizer(en_test_set['tokens'][1], is_split_into_words=True)
# tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

# len(en_test_set['tokens'][1]), len(tokenized_input["input_ids"])

In [18]:
# print(tokenized_input.word_ids())

In [19]:
# word_ids = tokenized_input.word_ids()
# aligned_labels = [-100 if i is None else example["ner_tags"][i] for i in word_ids]
# print(len(aligned_labels), len(tokenized_input["input_ids"]))

In [20]:
# print(word_ids)

In [71]:
label_all_tokens = True

The tokenizer adds special tokens at the beginning and the end of the tokens. Therefore, after tokenization, the length of tokens and ner_tags will be different. the following function solves the problem. By setting "label_all_tokens" to True will set ner_tag for the special tokens to -100 so that they are ignored in the loss function. Since in the multiNerd dataset, the sentences are already tokenized into words, the "is_split_into_words" is set to True to avoid errors.

In [72]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. the label is set to -100 so they are ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, set the label to either the current label or -100, depending on the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [23]:
# tokenize_and_align_labels(tag_test_set[:5])

The code below tokenizes and aligns the data that will be used in future steps.

In [24]:
tokenized_en_train_set = en_train_set.map(tokenize_and_align_labels, batched=True)
tokenized_en_validation_set = en_validation_set.map(tokenize_and_align_labels, batched=True)
tokenized_en_test_set = en_test_set.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/262560 [00:00<?, ? examples/s]

Map:   0%|          | 0/32820 [00:00<?, ? examples/s]

Map:   0%|          | 0/32908 [00:00<?, ? examples/s]

In [87]:
tokenized_tag_train_set = tag_train_set.map(tokenize_and_align_labels, batched=True)
tokenized_tag_validation_set = tag_validation_set.map(tokenize_and_align_labels, batched=True)
tokenized_tag_test_set = tag_test_set.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/267840 [00:00<?, ? examples/s]

Map:   0%|          | 0/33480 [00:00<?, ? examples/s]

Map:   0%|          | 0/33598 [00:00<?, ? examples/s]

In [26]:
tokenized_test_set = dataset["test"].map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/335986 [00:00<?, ? examples/s]

In [90]:
tokenized_sampled_test_set = sampled_test.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/33598 [00:00<?, ? examples/s]

In [76]:
label_list = list(tag_index.keys())
# print(label_list)

In [77]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Set *push_to_hub* yo False if not intending to push the fine-tuned model to hugging face repository afterwards, or skipped the login in to hugging face at the top of this notebook.

In [23]:
model_name = model_checkpoint.split("/")[-1]
systemA_args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_steps=5000,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [79]:
model_name = model_checkpoint.split("/")[-1]
systemB_args = TrainingArguments(
    f"{model_name}-finetuned-5tagOnly-{task}",
    evaluation_strategy = "epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_steps=5000,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=True,
)

In [80]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [81]:
metric = load_metric("seqeval")

<ipython-input-81-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [82]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [37]:
systemA_trainer = Trainer(
    model,
    systemA_args,
    train_dataset=tokenized_en_train_set,
    eval_dataset=tokenized_en_validation_set,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [38]:
systemA_trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.043000,0.075547,0.877783,0.877289,0.877536,0.974455
2,0.018100,0.100995,0.891282,0.875419,0.883279,0.975597
3,0.007500,0.131000,0.883609,0.893293,0.888425,0.976024


TrainOutput(global_step=98460, training_loss=0.03244236777425902, metrics={'train_runtime': 6538.2435, 'train_samples_per_second': 120.473, 'train_steps_per_second': 15.059, 'total_flos': 9653474211394320.0, 'train_loss': 0.03244236777425902, 'epoch': 3.0})

In [ ]:
systemA_trainer.evaluate()

In [ ]:
systemA_predictions, systemA_labels, _ = systemA_trainer.predict(tokenized_en_test_set) #tokenized_test_set
systemA_predictions = np.argmax(systemA_predictions, axis=2)

# Remove ignored index (special tokens)
systemA_true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(systemA_predictions, systemA_labels)
]
systemA_true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(systemA_predictions, systemA_labels)
]

systemA_results = metric.compute(predictions=systemA_true_predictions, references=systemA_true_labels)
systemA_results

In [39]:
systemA_trainer.push_to_hub()

events.out.tfevents.1701818306.4c0f47d93651.1491.0:   0%|          | 0.00/38.7k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/266M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

'https://huggingface.co/emilyblah/distilbert-base-uncased-finetuned-ner/tree/main/'

In [85]:
systemB_trainer = Trainer(
    model,
    systemB_args,
    train_dataset=tokenized_tag_train_set,
    eval_dataset=tokenized_tag_validation_set,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [86]:
systemB_trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.106100,0.104772,0.839279,0.756362,0.795666,0.968550
2,0.067300,0.097089,0.833046,0.811779,0.822275,0.971611
3,0.051900,0.101778,0.847209,0.820567,0.833675,0.973452


TrainOutput(global_step=100440, training_loss=0.08888037205691929, metrics={'train_runtime': 8673.5027, 'train_samples_per_second': 92.641, 'train_steps_per_second': 11.58, 'total_flos': 1.7837278056531744e+16, 'train_loss': 0.08888037205691929, 'epoch': 3.0})

In [88]:
systemB_trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/266M [00:00<?, ?B/s]

events.out.tfevents.1701877907.d8d1b8aa358d.1920.0:   0%|          | 0.00/39.3k [00:00<?, ?B/s]

'https://huggingface.co/emilyblah/distilbert-base-uncased-finetuned-5tagOnly-ner/tree/main/'

In [ ]:
systemB_trainer.evaluate()

In [89]:
systemB_predictions, systemB_labels, _ = systemB_trainer.predict(tokenized_tag_test_set) #tokenized_test_set
systemB_predictions = np.argmax(systemB_predictions, axis=2)

# Remove ignored index (special tokens)
systemB_true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(systemB_predictions, systemB_labels)
]
systemB_true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(systemB_predictions, systemB_labels)
]

systemB_results = metric.compute(predictions=systemB_true_predictions, references=systemB_true_labels)
systemB_results

{'ANIM': {'precision': 0.6118110236220472,
  'recall': 0.5987670177241202,
  'f1': 0.6052187459431391,
  'number': 7786},
 'DIS': {'precision': 0.5843986433602922,
  'recall': 0.5408015451472719,
  'f1': 0.561755485893417,
  'number': 4142},
 'LOC': {'precision': 0.8825530618462476,
  'recall': 0.8576856649395509,
  'f1': 0.8699416902324884,
  'number': 40530},
 'ORG': {'precision': 0.8020250868973855,
  'recall': 0.7527659574468085,
  'f1': 0.7766152045072072,
  'number': 7050},
 'PER': {'precision': 0.8896628073250611,
  'recall': 0.8876564801178203,
  'f1': 0.8886585113000206,
  'number': 21728},
 'overall_precision': 0.8372569089048106,
 'overall_recall': 0.8156236151459944,
 'overall_f1': 0.8262986911761954,
 'overall_accuracy': 0.9730126768039228}

In [91]:
systemB_predictions, systemB_labels, _ = systemB_trainer.predict(tokenized_sampled_test_set) #tokenized_test_set
systemB_predictions = np.argmax(systemB_predictions, axis=2)

# Remove ignored index (special tokens)
systemB_true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(systemB_predictions, systemB_labels)
]
systemB_true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(systemB_predictions, systemB_labels)
]

systemB_results = metric.compute(predictions=systemB_true_predictions, references=systemB_true_labels)
systemB_results

/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'ANIM': {'precision': 0.6118110236220472,
  'recall': 0.5987670177241202,
  'f1': 0.6052187459431391,
  'number': 7786},
 'BIO': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 37},
 'CEL': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 849},
 'DIS': {'precision': 0.5843986433602922,
  'recall': 0.5408015451472719,
  'f1': 0.561755485893417,
  'number': 4142},
 'EVE': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2853},
 'FOOD': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2465},
 'INST': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 193},
 'LOC': {'precision': 0.8825530618462476,
  'recall': 0.8576856649395509,
  'f1': 0.8699416902324884,
  'number': 40530},
 'MEDIA': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 4374},
 'MYTH': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 662},
 'ORG': {'precision': 0.8020250868973855,
  'recall': 0.7527659574468085,
  'f1': 0.7766152045072072,
  'number': 7050},
 'PER': {'precision': 0.8